<a href="https://colab.research.google.com/github/azzindani/03_Prompt_Engineering/blob/main/PE_EN_Qwen2_5_3B_Med_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importing Modules

In [1]:
!pip install -U -q bitsandbytes
!pip install -q optimum
!pip install -q auto-gptq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 424.1/424.1 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 31.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
   ━━━━

In [2]:
import torch

from transformers import (
  AutoTokenizer,
  AutoModelForCausalLM,
  GenerationConfig,
  BitsAndBytesConfig,
  GPTQConfig
)

if torch.cuda.is_available():
  print('GPU is available!')
else:
  print('GPU is not available.')

GPU is available!


## Import Model

In [3]:
model_name = 'Qwen/Qwen2.5-3B'

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [5]:
'''bnb_config = BitsAndBytesConfig(
  load_in_4bit = True,
  bnb_4bit_quant_type = 'nf4',
  bnb_4bit_compute_dtype = torch.float16,
  bnb_4bit_use_double_quant = True,
)

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  quantization_config = bnb_config,
  trust_remote_code = True
).to(device) #'''

"bnb_config = BitsAndBytesConfig(\n  load_in_4bit = True,\n  bnb_4bit_quant_type = 'nf4',\n  bnb_4bit_compute_dtype = torch.float16,\n  bnb_4bit_use_double_quant = True,\n)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n  model_name,\n  quantization_config = bnb_config,\n  trust_remote_code = True\n).to(device) #"

In [6]:
model = AutoModelForCausalLM.from_pretrained(
  model_name,
  torch_dtype = torch.float16,
  trust_remote_code = True
).to(device) #'''

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.6k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

## Import Tokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

## Model Preview

In [8]:
def assistant(prompt):
  prompt_format = f"### Human: {prompt}\n### Assistant :"

  inputs = tokenizer(
    prompt_format,
    return_tensors = 'pt'
  ).to(device)
  generation_config = GenerationConfig(
    do_sample = True,
    top_k = 1,
    temperature = 0.1,
    max_new_tokens = 1024,
    pad_token_id = tokenizer.eos_token_id
  )
  outputs = model.generate(
    **inputs,
    generation_config = generation_config
  )
  return print(tokenizer.decode(outputs[0], skip_special_tokens = True))

In [9]:
# Level 1: Basic Prompt
prompt = "What are the symptoms and treatments of diabetes?"
assistant(prompt)

### Human: What are the symptoms and treatments of diabetes?
### Assistant : Diabetes is a chronic condition that affects how your body regulates blood sugar levels. There are two main types of diabetes: type 1 and type 2. Type 1 diabetes is an autoimmune disorder where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is caused by insulin resistance, where the body's cells do not respond properly to insulin. Symptoms of diabetes include frequent urination, excessive thirst, fatigue, blurred vision, and unexplained weight loss. Treatment for diabetes includes lifestyle changes such as diet and exercise, as well as medication and insulin therapy.


In [10]:
# Level 2: Adding Detail
prompt = "Briefly describe the symptoms and treatments of diabetes."
assistant(prompt)

### Human: Briefly describe the symptoms and treatments of diabetes.
### Assistant : Diabetes is a chronic condition that occurs when the body is unable to produce or properly use insulin, a hormone that regulates blood sugar levels. Symptoms of diabetes include increased thirst, frequent urination, fatigue, and blurred vision. Treatment options include lifestyle changes, medication, and insulin therapy.


In [11]:
# Level 3: Specify Information Type and Audience
prompt = "Explain the common symptoms and treatments of diabetes to a general audience."
assistant(prompt)

### Human: Explain the common symptoms and treatments of diabetes to a general audience.
### Assistant : Diabetes is a chronic condition that affects the way the body processes glucose, or sugar, which is a primary source of energy for the body. There are two main types of diabetes: Type 1 and Type 2. Type 1 diabetes is an autoimmune disease where the body's immune system attacks and destroys the insulin-producing cells in the pancreas. Type 2 diabetes is caused by insulin resistance, where the body's cells do not respond properly to insulin, or a lack of insulin production. Symptoms of diabetes can include frequent urination, excessive thirst, fatigue, blurred vision, and unexplained weight loss. Treatment for diabetes typically involves a combination of lifestyle changes, such as diet and exercise, and medication, such as insulin or oral hypoglycemic agents. Regular monitoring of blood sugar levels is also important for managing diabetes.


In [12]:
# Level 4: Request for Structured Response
prompt = "List the main symptoms of diabetes, then outline the standard treatments."
assistant(prompt)

### Human: List the main symptoms of diabetes, then outline the standard treatments.
### Assistant : The main symptoms of diabetes are increased thirst, frequent urination, and unexplained weight loss. The standard treatments for diabetes include lifestyle changes, such as diet and exercise, and medication, such as insulin or oral hypoglycemic agents.


In [13]:
# Level 5: Differentiate by Type
prompt = "List the symptoms and treatments for both Type 1 and Type 2 diabetes."
assistant(prompt)

### Human: List the symptoms and treatments for both Type 1 and Type 2 diabetes.
### Assistant : Sure, here are the symptoms and treatments for both Type 1 and Type 2 diabetes:

**Type 1 Diabetes:**

Symptoms:
- Increased thirst and frequent urination
- Extreme hunger
- Unexplained weight loss
- Fatigue
- Blurred vision
- Slow-healing sores or frequent infections

Treatments:
- Insulin injections or an insulin pump
- Regular blood sugar monitoring
- Healthy eating habits
- Regular exercise
- Regular medical check-ups

**Type 2 Diabetes:**

Symptoms:
- Increased thirst and frequent urination
- Extreme hunger
- Unexplained weight loss
- Fatigue
- Blurred vision
- Slow-healing sores or frequent infections
- Tingling or numbness in the hands or feet
- Frequent infections

Treatments:
- Healthy eating habits
- Regular exercise
- Weight loss if overweight
- Oral medications
- Insulin injections or an insulin pump
- Regular blood sugar monitoring
- Regular medical check-ups

It's important to

In [14]:
# Level 6: Target a Professional Audience
prompt = "Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know."
assistant(prompt)

### Human: Describe the symptoms, diagnostic criteria, and treatment options for Type 1 and Type 2 diabetes, focusing on what healthcare providers should know.
### Assistant : Type 1 diabetes is an autoimmune disease in which the body's immune system attacks and destroys the insulin-producing beta cells in the pancreas. This leads to a complete lack of insulin production, resulting in high blood sugar levels. The symptoms of Type 1 diabetes include frequent urination, excessive thirst, fatigue, weight loss, and blurred vision. The diagnostic criteria for Type 1 diabetes include a fasting blood glucose level of 126 mg/dL or higher, or a random blood glucose level of 200 mg/dL or higher with symptoms of hyperglycemia. Treatment options for Type 1 diabetes include insulin therapy, which involves injecting insulin multiple times a day to maintain blood sugar levels within a target range. Healthcare providers should also educate patients on self-monitoring blood glucose levels, carbohydrate

In [15]:
# Level 7: Clinical Detail with Disease Stages
prompt = "Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages."
assistant(prompt)

### Human: Explain the symptoms, diagnostic markers, and treatments for diabetes, distinguishing between prediabetes, Type 1, and Type 2 diabetes. Include early signs, progression, and advanced stages.
### Assistant : Diabetes is a metabolic disorder characterized by high blood sugar levels. It can be classified into three main types: prediabetes, Type 1 diabetes, and Type 2 diabetes. Each type has distinct symptoms, diagnostic markers, and treatments.

#### Prediabetes
- **Symptoms**: Prediabetes often has no noticeable symptoms. However, some individuals may experience increased thirst, frequent urination, and fatigue.
- **Diagnostic Markers**: Prediabetes is diagnosed through blood tests that measure fasting blood glucose levels or HbA1c (glycated hemoglobin) levels. A fasting blood glucose level between 100-125 mg/dL or an HbA1c between 5.7-6.4% indicates prediabetes.
- **Treatments**: Lifestyle changes such as weight loss, regular exercise, and a healthy diet can help prevent or d

In [16]:
# Level 8: Add Nuance with Treatment Personalization
prompt = "Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors."
assistant(prompt)

### Human: Describe the symptoms, diagnostic markers, and treatment options for diabetes, with a focus on Type 1 and Type 2. Include how treatments may vary based on patient age, co-existing conditions, and lifestyle factors.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels. It can be classified into two main types: Type 1 and Type 2. Each type has distinct symptoms, diagnostic markers, and treatment options.

#### Type 1 Diabetes
**Symptoms:**
- Frequent urination
- Extreme thirst
- Extreme hunger
- Unexplained weight loss
- Fatigue
- Blurred vision
- Slow-healing sores
- Frequent infections

**Diagnostic Markers:**
- High blood sugar levels
- Low insulin levels
- Autoantibodies against pancreatic beta cells

**Treatment Options:**
- **Insulin Therapy:** Essential for managing blood sugar levels. Patients may need multiple daily injections or use an insulin pump.
- **Lifestyle Changes:** Regular exercise, healthy diet, and weight manage

In [17]:
# Level 9: Consider Risks, Complications, and Treatment Impacts
prompt = "Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment."
assistant(prompt)

### Human: Provide a comprehensive overview of diabetes, covering symptoms, diagnostic criteria, and treatment options for both Type 1 and Type 2 diabetes. Explain how risks and complications, such as neuropathy and cardiovascular disease, may impact treatment decisions. Include possible side effects and patient adherence concerns for each treatment.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to insulin resistance or insufficient insulin production. It can be broadly classified into two types: Type 1 and Type 2.

#### Type 1 Diabetes
**Symptoms:**
- Frequent urination
- Increased thirst
- Extreme hunger
- Unexplained weight loss
- Fatigue
- Blurred vision
- Slow healing of cuts and bruises

**Diagnostic Criteria:**
- Fasting plasma glucose (FPG) ≥ 126 mg/dL or random plasma glucose ≥ 200 mg/dL
- Glycated hemoglobin (HbA1c) ≥ 6.5%

**Treatment Options:**
1. **Insulin Therapy:** Essential for all patients with Type 1 diabetes. In

In [18]:
# Level 10: Advanced Detail for Clinical Research or Specialist Training
prompt = "Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes."
assistant(prompt)

### Human: Provide an in-depth, evidence-based analysis of the symptoms, diagnostic criteria, and treatment approaches for diabetes, including both Type 1 and Type 2. Discuss the pathophysiology, biomarkers, and clinical guidelines, as well as how complications like neuropathy, retinopathy, and cardiovascular disease alter treatment. Include recent clinical research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.
### Assistant : Diabetes is a chronic metabolic disorder characterized by high blood sugar levels due to insulin resistance or insufficient insulin production. It can be classified into two main types: Type 1 and Type 2. This analysis will cover the symptoms, diagnostic criteria, treatment approaches, pathophysiology, biomarkers, clinical guidelines, complications, recent research findings, new pharmaceutical options, and the role of lifestyle interventions in managing long-term patient outcomes.

#### Sympt